In [1]:
import numpy as np
import pandas as pd

from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

### 1. Choose a transactional dataset which is suitable for association rule analysis problem from the UCI Machine Learning Repository or from any other dataset repositories.
### 2. Download the selected dataset – you are required to provide a link to the download page for your selected dataset.
The dataset can be found [here](https://archive.ics.uci.edu/ml/datasets/Online+Retail+II)

In [2]:

df = pd.read_excel('dataset/online_retail_II.xlsx', engine='openpyxl')

In [4]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


### 3. Describe the dataset and the data mining task.
The dataset is a transnational data set that contains online retail transactions. The task is to build the rules to uncover associations between.

In [5]:
print(f'The dataset containe recorders start form {pd.to_datetime(df.InvoiceDate.min()).date() } to {pd.to_datetime(df.InvoiceDate.max()).date() } and has {df.StockCode.nunique()} unique items')

The dataset containe recorders start form 2009-12-01 to 2010-12-09 and has 4632 unique items


### 4. Display the number of instances.
### 5. Display the number of attributes.


In [6]:
print(f"The data has {df.shape[0]} records, {df.shape[1]} attributes")

The data has 525461 records, 8 attributes


### 6. Display a statistical summary for all the attributes.

In [7]:
df.describe()

,Quantity,Price,Customer ID
count,525461.000000,525461.000000,417534.000000
mean,10.337667,4.688834,15360.645478
std,107.424110,146.126914,1680.811316
min,-9600.000000,-53594.360000,12346.000000
25%,1.000000,1.250000,13983.000000
50%,3.000000,2.100000,15311.000000
75%,10.000000,4.210000,16799.000000
max,19152.000000,25111.090000,18287.000000


### 7. Check whether the selected dataset has any data quality issues and choose suitable strategies to deal with any issue (if exists). 
The dataset apparently doesn't have any issue but for making sure, the rows that don't contain inovice number are dropped

In [8]:
# Dropping rows that doesn't contain invoice number
df.dropna(inplace=True)
df = df[~df['Invoice'].str.contains('C', na=False)]

In this dataset, InvoiceNo means the orderIDNo, and the StockCode represents different kinds of products. In order to reorgnize this dataset, orders rearranged as rows and products being columns.

In [9]:
df = df.groupby(['Invoice','Description'])['Quantity'].sum().unstack().fillna(0)

In [10]:
# replace all values of quantity >=1 by 1
def encoding(x):
    if x <= 0:
        return 0
    else:
        return 1

basket_final = df.applymap(encoding)

In [11]:
# drop "POSTAGE" since it's not a real product.
basket_final.drop('POSTAGE', inplace=True, axis=1)

basket_final.head()

Description,DOORMAT UNION JACK GUNS AND ROSES,3 STRIPEY MICE FELTCRAFT,4 PURPLE FLOCK DINNER CANDLES,ANIMAL STICKERS,BLACK PIRATE TREASURE CHEST,BROWN PIRATE TREASURE CHEST,Bank Charges,CAMPHOR WOOD PORTOBELLO MUSHROOM,CHERRY BLOSSOM DECORATIVE FLASK,FAIRY CAKE CANDLES,...,ZINC HEART LATTICE CHARGER LARGE,ZINC HEART LATTICE CHARGER SMALL,ZINC HEART LATTICE DOUBLE PLANTER,ZINC HEART LATTICE PLANTER BOWL,ZINC HEART LATTICE T-LIGHT HOLDER,ZINC HEART LATTICE TRAY OVAL,ZINC METAL HEART DECORATION,ZINC POLICE BOX LANTERN,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK
Invoice,,,,,,,,,,,,,,,,,,,,,
489434,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489435,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489436,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489437,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
489438,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 8. Generate Rules using Apriori Algorithm 


In [12]:
## Apriori to select the most important itemsets
frequent_itemsets = apriori(basket_final, min_support = 0.02, use_colnames = True)

In [13]:
rules = association_rules(frequent_itemsets, metric = "lift", min_threshold = 1)
rules.sort_values('lift',ascending = False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
29,(WOODEN FRAME ANTIQUE WHITE ),(WOODEN PICTURE FRAME WHITE FINISH),0.052511,0.042571,0.028832,0.549058,12.897504,0.026596,2.123178
28,(WOODEN PICTURE FRAME WHITE FINISH),(WOODEN FRAME ANTIQUE WHITE ),0.042571,0.052511,0.028832,0.677262,12.897504,0.026596,2.935780
25,(SWEETHEART CERAMIC TRINKET BOX),(STRAWBERRY CERAMIC TRINKET BOX),0.042050,0.069477,0.032371,0.769802,11.079959,0.029449,4.042272
24,(STRAWBERRY CERAMIC TRINKET BOX),(SWEETHEART CERAMIC TRINKET BOX),0.069477,0.042050,0.032371,0.465918,11.079959,0.029449,1.793636
6,(HOT WATER BOTTLE TEA AND SYMPATHY),(CHOCOLATE HOT WATER BOTTLE),0.043768,0.041686,0.020088,0.458977,11.010301,0.018264,1.771301
7,(CHOCOLATE HOT WATER BOTTLE),(HOT WATER BOTTLE TEA AND SYMPATHY),0.041686,0.043768,0.020088,0.481898,11.010301,0.018264,1.845643
8,(HEART OF WICKER LARGE),(HEART OF WICKER SMALL),0.051887,0.045485,0.025553,0.492477,10.827178,0.023193,1.880734
9,(HEART OF WICKER SMALL),(HEART OF WICKER LARGE),0.045485,0.051887,0.025553,0.561785,10.827178,0.023193,2.163580
19,(LUNCH BAG SPACEBOY DESIGN ),(LUNCH BAG SUKI DESIGN ),0.047307,0.049649,0.023627,0.499450,10.059676,0.021279,1.898614
18,(LUNCH BAG SUKI DESIGN ),(LUNCH BAG SPACEBOY DESIGN ),0.049649,0.047307,0.023627,0.475891,10.059676,0.021279,1.817739


In [14]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(PACK OF 60 PINK PAISLEY CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.048296,0.059120,0.024356,0.504310,8.530214,0.021501,1.898122
1,(60 TEATIME FAIRY CAKE CASES),(PACK OF 60 PINK PAISLEY CAKE CASES),0.059120,0.048296,0.024356,0.411972,8.530214,0.021501,1.618467
2,(PACK OF 72 RETRO SPOT CAKE CASES),(60 TEATIME FAIRY CAKE CASES),0.060370,0.059120,0.024876,0.412069,6.969987,0.021307,1.600323
3,(60 TEATIME FAIRY CAKE CASES),(PACK OF 72 RETRO SPOT CAKE CASES),0.059120,0.060370,0.024876,0.420775,6.969987,0.021307,1.622219
4,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.157221,0.069373,0.020297,0.129096,1.860905,0.009390,1.068576


From the result above, I observe that:

1. **PACK OF 60 PINK PAISLEY CAKE CASES** and **60 TEATIME FAIRY CAKE CASES** are purchased together.
2. **PACK OF 72 RETRO SPOT CAKE CASES** and **60 TEATIME FAIRY CAKE CASES** are purchased together.
3. **WHITE HANGING HEART T-LIGHT HOLDER** and **ASSORTED COLOUR BIRD ORNAMENT** are purchased together.
